In [17]:
import pandas as pd
import numpy as np
import random

## A. Melburne Housing Dataset

### A.1 General

Originally created by *Tony Pino*

License CC BY-NC-SA 4.0

Source: https://www.kaggle.com/anthonypino/melbourne-housing-market

Downloaded date 2022/1/30

### A.2 Data dictionary

**Suburb**: Suburb name

**Address**: Address

**Rooms**: Number of rooms

**Price**: Price in Australian dollars

**Method**:
S - property sold;
SP - property sold prior;
PI - property passed in;
PN - sold prior not disclosed;
SN - sold not disclosed;
NB - no bid;
VB - vendor bid;
W - withdrawn prior to auction;
SA - sold after auction;
SS - sold after auction price not disclosed.
N/A - price or highest bid not available.

**Type**:
br - bedroom(s);
h - house,cottage,villa, semi,terrace;
u - unit, duplex;
t - townhouse;
dev site - development site;
o res - other residential.

**SellerG**: Real Estate Agent

**Date**: Date sold

**Distance**: Distance from CBD in Kilometres

**Regionname**: General Region (West, North West, North, North east …etc)

**Propertycount**: Number of properties that exist in the suburb.

**Bedroom2** : Scraped # of Bedrooms (from different source)

**Bathroom**: Number of Bathrooms

**Car**: Number of carspots

**Landsize**: Land Size in Metres

**BuildingArea**: Building Size in Metres

**YearBuilt**: Year the house was built

**CouncilArea**: Governing council for the area

**Lattitude**: Self explanitory

**Longtitude**: Self explanitory

### A.3 Modifications to the datset for the purposes of this book

#### Missing values detection
In the original `Melbourne housing` dataset, all missing values are "empty", which is read-in correctly as `NaN` by `pandas`.

The following modifications to the original dataset were performed for the purpose of learning how to detect poorly encoded missing values.

1. Missing values (numerical or categorical features) are encoded in some way described in the data dictionary : **Bathroom** and **Postcode**
2. Missing values of numeric feature are encoded as string : **Bedroom2**
3. Missing values in numeric feature are encoded as numerical (eg -9999) : **Price**
4. Missing values in string/categorical column are encoded in a way that is not described in the data dictionary : **Type**


In [86]:
df = pd.read_csv('Melbourne_housing_FULL.csv')
df.loc[[400,22000, 3000,1234],'Date']=np.nan

df.replace({'Bathroom':np.nan, 'Postcode':np.nan, 'Bedroom2': np.nan, 'Price':np.nan, 'Type':np.nan},
          {'Bathroom':'notavailable', 'Postcode':'missing', 'Bedroom2':'-', 'Price':-9999, 'Type':'UNK'},
          inplace = True)

df.to_csv('Melbourne_housing_Modified.csv',index=False)

In [100]:
housing = pd.read_csv("Melbourne_housing_Modified.csv")
hs = housing.applymap(lambda x: np.nan if x in ['notavailable','missing','-','UNK',-9999] else x)

#hs.to_csv('Melbourne_housing_nan.csv',index=False)

/usr/local/Cellar/python/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [103]:
hs.dtypes

Suburb            object
Address           object
Rooms              int64
Type              object
Price            float64
Method            object
SellerG           object
Date              object
Distance         float64
Postcode          object
Bedroom2          object
Bathroom          object
Car              float64
Landsize         float64
BuildingArea     float64
YearBuilt        float64
CouncilArea       object
Lattitude        float64
Longtitude       float64
Regionname        object
Propertycount    float64
dtype: object

In [104]:
a = hs.astype(flo)

ValueError: could not convert string to float: 'Western Metropolitan'

In [92]:
hs = pd.read_csv("Melbourne_housing_nan.csv")
df = pd.read_csv("Melbourne_housing_FULL.csv")

In [88]:
def missing_detection(df):
    columns = df.columns
    other_encoding = {'Method': ['N/A']}
    categorical_values = {'Type' : ['br','h','u','t','dev site','o res'],
                   'Method': ['S','SP','PI', 'PN','SN','NB','VB','W','SA','SS']}
    return pd.DataFrame(data ={'col_name' :columns, 
                    'datatype' : [np.dtype(df[col]) for col in columns],
                    'col_sample' : [df.loc[10,col] for col in columns],
                    'pct_detected_nan' : [round((df[col].isnull().sum())*100/df.shape[0],4) for col in columns],
                    'max_value': [np.max(df[col]) if np.dtype(df[col])!=object else "NA" for col in columns ],
                    'min_value': [np.min(df[col]) if np.dtype(df[col])!=object else "NA" for col in columns ],
                    'pct_encoded_nan' : [round(sum(df[col].isin(other_encoding[col]))*100/df.shape[0],4) if col in other_encoding.keys() else "NA" for col in columns],
                    'pct_nonecoded_nan' : [round((df.shape[0]-sum(df[col].isin(categorical_values[col])))/df.shape[0]*100/df.shape[0],4) if col in categorical_values.keys() else "NA" for col in columns]
                       })
      

In [98]:
missing_detection(hs)

,col_name,col_sample,datatype,max_value,min_value,pct_detected_nan,pct_encoded_nan,pct_nonecoded_nan
0,Suburb,Abbotsford,object,NA,NA,0.0000,NA,NA
1,Address,129 Charles St,object,NA,NA,0.0000,NA,NA
2,Rooms,2,int64,16,1,0.0000,NA,NA
3,Type,h,object,NA,NA,0.0000,NA,0
4,Price,941000,float64,1.12e+07,85000,21.8321,NA,NA
5,Method,S,object,NA,NA,0.0000,0,0
6,SellerG,Jellis,object,NA,NA,0.0000,NA,NA
7,Date,7/05/2016,object,NA,NA,0.0115,NA,NA
8,Distance,2.5,float64,48.1,0,0.0029,NA,NA
9,Postcode,3067,float64,3978,3000,0.0029,NA,NA


In [99]:
missing_detection(df)

,col_name,col_sample,datatype,max_value,min_value,pct_detected_nan,pct_encoded_nan,pct_nonecoded_nan
0,Suburb,Abbotsford,object,NA,NA,0.0000,NA,NA
1,Address,129 Charles St,object,NA,NA,0.0000,NA,NA
2,Rooms,2,int64,16,1,0.0000,NA,NA
3,Type,h,object,NA,NA,0.0000,NA,0
4,Price,941000,float64,1.12e+07,85000,21.8321,NA,NA
5,Method,S,object,NA,NA,0.0000,0,0
6,SellerG,Jellis,object,NA,NA,0.0000,NA,NA
7,Date,7/05/2016,object,NA,NA,0.0000,NA,NA
8,Distance,2.5,float64,48.1,0,0.0029,NA,NA
9,Postcode,3067,float64,3978,3000,0.0029,NA,NA


In [68]:
df2 = pd.read_csv("Melbourne_housing_Modified.csv")

Is dtype agreeing with what we expect to be automatically detected
Are the categorical values the ones we expect
Are the numerical values within a range that makes sense
Are there other detected missing values

In [76]:
for col in df2.columns:
    a={df2.loc[0:2,col].values, np.dtype(df2[col]),round((df2[col].isnull().sum()))} 

TypeError: unhashable type: 'numpy.ndarray'

In [7]:
for col in df.columns:
    print(col, 
          df.loc[0:4,col].values,
          np.dtype(df[col]), 
          round((df[col].isnull().sum())*100/df.shape[0],4))

Suburb ['Abbotsford' 'Abbotsford' 'Abbotsford' 'Abbotsford' 'Abbotsford'] object 0.0
Address ['68 Studley St' '85 Turner St' '25 Bloomburg St' '18/659 Victoria St'
 '5 Charles St'] object 0.0
Rooms [2 2 2 3 3] int64 0.0
Type ['h' 'h' 'h' 'u' 'h'] object 0.0
Price [      nan  1480000.  1035000.       nan  1465000.] float64 21.8321
Method ['SS' 'S' 'S' 'VB' 'SP'] object 0.0
SellerG ['Jellis' 'Biggin' 'Biggin' 'Rounds' 'Biggin'] object 0.0
Date ['3/09/2016' '3/12/2016' '4/02/2016' '4/02/2016' '4/03/2017'] object 0.0
Distance [ 2.5  2.5  2.5  2.5  2.5] float64 0.0029
Postcode [ 3067.  3067.  3067.  3067.  3067.] float64 0.0029
Bedroom2 [ 2.  2.  2.  3.  3.] float64 23.5735
Bathroom [ 1.  1.  1.  2.  2.] float64 23.5993
Car [ 1.  1.  0.  1.  0.] float64 25.0394
Landsize [ 126.  202.  156.    0.  134.] float64 33.8813
BuildingArea [  nan   nan   79.   nan  150.] float64 60.5761
YearBuilt [   nan    nan  1900.    nan  1900.] float64 55.3863
CouncilArea ['Yarra City Council' 'Yarra City Counci

In [13]:
round((df.isnull().sum())*100/df.shape[0],2)

Suburb            0.00
Address           0.00
Rooms             0.00
Type              0.00
Price            21.83
Method            0.00
SellerG           0.00
Date              0.00
Distance          0.00
Postcode          0.00
Bedroom2         23.57
Bathroom         23.60
Car              25.04
Landsize         33.88
BuildingArea     60.58
YearBuilt        55.39
CouncilArea       0.01
Lattitude        22.88
Longtitude       22.88
Regionname        0.01
Propertycount     0.01
dtype: float64

In [27]:
print(np.unique(df['Price']))

[  85000.  112000.  121000. ...,      nan      nan      nan]
